In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_beads.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation:   0%|                             | 0/30 [00:00<?, ?it/s]/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))
Eval image generation: 100%|████████████████████| 30/30 [00:02<00:00, 11.52it/s]


eval images shape: (30, 1041, 1302) contain 2909 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
model.simulation_params

{'train_size': 128,
 'train_type': 'uniform',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.15,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'bg_width': 20,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=10000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 10849.972 || Time Upd.: 648.6 ms  || BatchNr.: 100
 || Cost: 1018.290 || Time Upd.: 626.9 ms  || BatchNr.: 200
 || Cost: 502.361 || Time Upd.: 617.2 ms  || BatchNr.: 300
 || Cost: 434.991 || Time Upd.: 622.8 ms  || BatchNr.: 400
 || Cost: 389.988 || Time Upd.: 626.2 ms  || BatchNr.: 500
 || Cost: 294.279 || Time Upd.: 633.3 ms  || BatchNr.: 600
 || Cost: 301.545 || Time Upd.: 638.5 ms  || BatchNr.: 700
 || Cost: 282.614 || Time Upd.: 631.2 ms  || BatchNr.: 800
 || Cost: 250.030 || Time Upd.: 656.5 ms  || BatchNr.: 900
 || Cost: 212.810 || Time Upd.: 634.0 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 3026FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2989 ,preds: 2939
JoR: 2.785 || Eff_3d: 61.947 || Jaccard: 0.978 || Factor: 127.749 || RMSE_lat: 35.120 || RMSE_ax: 81.717 || RMSE_x:

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2887
JoR: 4.441 || Eff_3d: 69.868 || Jaccard: 0.993 || Factor: 103.035 || RMSE_lat: 22.371 || RMSE_ax: 75.756 || RMSE_x: 15.441 || RMSE_y: 16.188 || Cost: 99.646 || Recall: 0.997 || Precision: 0.997 || BatchNr.: 3400 || Time Upd.: 641.5 ms 
 || Count Cost: 109.0483 || Localization Cost: 51.4185 || Bg Cost: 0.0862 || P_locs Cost: 78.0743saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 69.868 || 22.371 || 75.756 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2865
JoR: 4.500 || Eff_3d: 74.632 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2880
JoR: 4.728 || Eff_3d: 75.028 || Jaccard: 0.995 || Factor: 100.013 || RMSE_lat: 21.047 || RMSE_ax: 57.775 || RMSE_x: 14.341 || RMSE_y: 15.404 || Cost: 91.775 || Recall: 0.996 || Precision: 0.999 || BatchNr.: 4600 || Time Upd.: 641.0 ms 
 || Count Cost: 107.0371 || Localization Cost: 44.0701 || Bg Cost: 0.0924 || P_locs Cost: 75.6357saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 75.028 || 21.047 || 57.775 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2885
JoR: 4.630 || Eff_3d: 76.569 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2889
JoR: 4.575 || Eff_3d: 76.952 || Jaccard: 0.997 || Factor: 110.275 || RMSE_lat: 21.791 || RMSE_ax: 48.604 || RMSE_x: 14.749 || RMSE_y: 16.041 || Cost: 88.581 || Recall: 0.999 || Precision: 0.998 || BatchNr.: 5800 || Time Upd.: 643.0 ms 
 || Count Cost: 107.6532 || Localization Cost: 41.5379 || Bg Cost: 0.0858 || P_locs Cost: 74.5879saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 76.952 || 21.791 || 48.604 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2883
JoR: 4.485 || Eff_3d: 76.486 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2877
JoR: 4.918 || Eff_3d: 78.517 || Jaccard: 0.995 || Factor: 96.298 || RMSE_lat: 20.226 || RMSE_ax: 45.454 || RMSE_x: 13.869 || RMSE_y: 14.722 || Cost: 78.823 || Recall: 0.995 || Precision: 0.999 || BatchNr.: 7000 || Time Upd.: 635.1 ms 
 || Count Cost: 107.2843 || Localization Cost: 34.0206 || Bg Cost: 0.0599 || P_locs Cost: 74.2200saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 78.517 || 20.226 || 45.454 || 7000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2882
JoR: 4.810 || Eff_3d: 77.854 || 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2890
JoR: 4.800 || Eff_3d: 78.040 || Jaccard: 0.995 || Factor: 105.830 || RMSE_lat: 20.734 || RMSE_ax: 46.353 || RMSE_x: 14.037 || RMSE_y: 15.260 || Cost: 75.131 || Recall: 0.998 || Precision: 0.997 || BatchNr.: 8200 || Time Upd.: 639.2 ms 
 || Count Cost: 108.4868 || Localization Cost: 30.1801 || Bg Cost: 0.0635 || P_locs Cost: 72.7068saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 78.040 || 20.734 || 46.353 || 8200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2881
JoR: 4.926 || Eff_3d: 78.641 ||

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2884
JoR: 4.865 || Eff_3d: 79.392 || Jaccard: 0.997 || Factor: 101.315 || RMSE_lat: 20.497 || RMSE_ax: 41.430 || RMSE_x: 14.109 || RMSE_y: 14.869 || Cost: 69.251 || Recall: 0.998 || Precision: 0.999 || BatchNr.: 9400 || Time Upd.: 633.7 ms 
 || Count Cost: 105.9605 || Localization Cost: 24.8146 || Bg Cost: 0.0654 || P_locs Cost: 72.0374saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 79.392 || 20.497 || 41.430 || 9400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 2923FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 2888 ,preds: 2869
JoR: 4.902 || Eff_3d: 77.201 ||

In [12]:
1+1

2

In [13]:
plot_train_record(model)